#Bag of Words en PySparck
```
Estudiante: Charlie Joel Luna Ccasani
Código    : 161368
Lugar     : Cusco, Perú, 2022.
Propósito : PySpark:
```

##Preparamos el entorno de desarrollo

In [1]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 39 kB/s 
     |████████████████████████████████| 198 kB 48.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=9c59e158a041d108dffc0652ded7e1af3a47e7743aaab4a533e6cd21ac34aabd
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [2]:
from pyspark import SparkContext, SparkConf
conf=SparkConf().setAppName("Tarea_Spark")
sc=SparkContext(conf=conf)

##1. Term-Frequency

###Crearemos un conjunto de datos para el ejemplo (todo en minúsculas y en inglés)

In [3]:
data=[(1,'Yo amo mi gato'),(2,"Yo odio mi gato pero me gusta cantar"),(3,"Amo cantar y me encanta hacerlo")]
lines=sc.parallelize(data)
lines.collect()

[(1, 'Yo amo mi gato'),
 (2, 'Yo odio mi gato pero me gusta cantar'),
 (3, 'Amo cantar y me encanta hacerlo')]

###Asignamos pares de claves a través de la función FlatMap para enlistarlos

In [4]:
map1=lines.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])
map1.collect()

[((1, 'Yo'), 1),
 ((1, 'amo'), 1),
 ((1, 'mi'), 1),
 ((1, 'gato'), 1),
 ((2, 'Yo'), 1),
 ((2, 'odio'), 1),
 ((2, 'mi'), 1),
 ((2, 'gato'), 1),
 ((2, 'pero'), 1),
 ((2, 'me'), 1),
 ((2, 'gusta'), 1),
 ((2, 'cantar'), 1),
 ((3, 'Amo'), 1),
 ((3, 'cantar'), 1),
 ((3, 'y'), 1),
 ((3, 'me'), 1),
 ((3, 'encanta'), 1),
 ((3, 'hacerlo'), 1)]

###Reducimos los pares encontrando para obtener su frecuencia

In [5]:
reduce=map1.reduceByKey(lambda x,y:x+y)
reduce.collect()

[((1, 'Yo'), 1),
 ((1, 'amo'), 1),
 ((1, 'gato'), 1),
 ((2, 'odio'), 1),
 ((2, 'mi'), 1),
 ((2, 'pero'), 1),
 ((2, 'gusta'), 1),
 ((2, 'cantar'), 1),
 ((3, 'Amo'), 1),
 ((3, 'me'), 1),
 ((3, 'encanta'), 1),
 ((1, 'mi'), 1),
 ((2, 'Yo'), 1),
 ((2, 'gato'), 1),
 ((2, 'me'), 1),
 ((3, 'cantar'), 1),
 ((3, 'y'), 1),
 ((3, 'hacerlo'), 1)]

###Organizamos los datos mapeando en una función lamda

In [6]:
tf=reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))
tf.collect()

[('Yo', (1, 1)),
 ('amo', (1, 1)),
 ('gato', (1, 1)),
 ('odio', (2, 1)),
 ('mi', (2, 1)),
 ('pero', (2, 1)),
 ('gusta', (2, 1)),
 ('cantar', (2, 1)),
 ('Amo', (3, 1)),
 ('me', (3, 1)),
 ('encanta', (3, 1)),
 ('mi', (1, 1)),
 ('Yo', (2, 1)),
 ('gato', (2, 1)),
 ('me', (2, 1)),
 ('cantar', (3, 1)),
 ('y', (3, 1)),
 ('hacerlo', (3, 1))]

##2. Cálculo del IDF

###Mapeamos los token con su respectivo identificador TF

In [7]:
map3=reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
map3.collect()

[('Yo', (1, 1, 1)),
 ('amo', (1, 1, 1)),
 ('gato', (1, 1, 1)),
 ('odio', (2, 1, 1)),
 ('mi', (2, 1, 1)),
 ('pero', (2, 1, 1)),
 ('gusta', (2, 1, 1)),
 ('cantar', (2, 1, 1)),
 ('Amo', (3, 1, 1)),
 ('me', (3, 1, 1)),
 ('encanta', (3, 1, 1)),
 ('mi', (1, 1, 1)),
 ('Yo', (2, 1, 1)),
 ('gato', (2, 1, 1)),
 ('me', (2, 1, 1)),
 ('cantar', (3, 1, 1)),
 ('y', (3, 1, 1)),
 ('hacerlo', (3, 1, 1))]

###Extraemos los tokens

In [8]:
map4=map3.map(lambda x:(x[0],x[1][2]))
map4.collect()

[('Yo', 1),
 ('amo', 1),
 ('gato', 1),
 ('odio', 1),
 ('mi', 1),
 ('pero', 1),
 ('gusta', 1),
 ('cantar', 1),
 ('Amo', 1),
 ('me', 1),
 ('encanta', 1),
 ('mi', 1),
 ('Yo', 1),
 ('gato', 1),
 ('me', 1),
 ('cantar', 1),
 ('y', 1),
 ('hacerlo', 1)]

###Reducimos los pares

In [9]:
reduce2=map4.reduceByKey(lambda x,y:x+y)
reduce2.collect()

[('odio', 1),
 ('mi', 2),
 ('pero', 1),
 ('gusta', 1),
 ('cantar', 2),
 ('y', 1),
 ('hacerlo', 1),
 ('Yo', 2),
 ('amo', 1),
 ('gato', 2),
 ('Amo', 1),
 ('me', 2),
 ('encanta', 1)]

###Calculamos la putuación IDF

In [10]:
import math
from pyspark.sql.functions import *
idf=reduce2.map(lambda x: (x[0],math.log10(len(data)/x[1])))
idf.collect()

[('odio', 0.47712125471966244),
 ('mi', 0.17609125905568124),
 ('pero', 0.47712125471966244),
 ('gusta', 0.47712125471966244),
 ('cantar', 0.17609125905568124),
 ('y', 0.47712125471966244),
 ('hacerlo', 0.47712125471966244),
 ('Yo', 0.17609125905568124),
 ('amo', 0.47712125471966244),
 ('gato', 0.17609125905568124),
 ('Amo', 0.47712125471966244),
 ('me', 0.17609125905568124),
 ('encanta', 0.47712125471966244)]

###Juntamos las puntuaciones del TF e IDF

In [11]:
rdd=tf.join(idf)
rdd.collect()

[('cantar', ((2, 1), 0.17609125905568124)),
 ('cantar', ((3, 1), 0.17609125905568124)),
 ('y', ((3, 1), 0.47712125471966244)),
 ('hacerlo', ((3, 1), 0.47712125471966244)),
 ('Yo', ((1, 1), 0.17609125905568124)),
 ('Yo', ((2, 1), 0.17609125905568124)),
 ('gato', ((1, 1), 0.17609125905568124)),
 ('gato', ((2, 1), 0.17609125905568124)),
 ('encanta', ((3, 1), 0.47712125471966244)),
 ('odio', ((2, 1), 0.47712125471966244)),
 ('mi', ((2, 1), 0.17609125905568124)),
 ('mi', ((1, 1), 0.17609125905568124)),
 ('pero', ((2, 1), 0.47712125471966244)),
 ('gusta', ((2, 1), 0.47712125471966244)),
 ('amo', ((1, 1), 0.47712125471966244)),
 ('Amo', ((3, 1), 0.47712125471966244)),
 ('me', ((3, 1), 0.17609125905568124)),
 ('me', ((2, 1), 0.17609125905568124))]

###Finalmente, ordenamos los datos con la puntuación TF e IDF

In [12]:
rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()
rdd.collect()

[(1, ('Yo', 1, 0.17609125905568124, 0.17609125905568124)),
 (1, ('gato', 1, 0.17609125905568124, 0.17609125905568124)),
 (1, ('mi', 1, 0.17609125905568124, 0.17609125905568124)),
 (1, ('amo', 1, 0.47712125471966244, 0.47712125471966244)),
 (2, ('cantar', 1, 0.17609125905568124, 0.17609125905568124)),
 (2, ('Yo', 1, 0.17609125905568124, 0.17609125905568124)),
 (2, ('gato', 1, 0.17609125905568124, 0.17609125905568124)),
 (2, ('odio', 1, 0.47712125471966244, 0.47712125471966244)),
 (2, ('mi', 1, 0.17609125905568124, 0.17609125905568124)),
 (2, ('pero', 1, 0.47712125471966244, 0.47712125471966244)),
 (2, ('gusta', 1, 0.47712125471966244, 0.47712125471966244)),
 (2, ('me', 1, 0.17609125905568124, 0.17609125905568124)),
 (3, ('cantar', 1, 0.17609125905568124, 0.17609125905568124)),
 (3, ('y', 1, 0.47712125471966244, 0.47712125471966244)),
 (3, ('hacerlo', 1, 0.47712125471966244, 0.47712125471966244)),
 (3, ('encanta', 1, 0.47712125471966244, 0.47712125471966244)),
 (3, ('Amo', 1, 0.477121254